In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from codecarbon import EmissionsTracker
from ptflops import get_model_complexity_info
import os
import pandas as pd
import gc

In [2]:
# ----------------------------
# Clean GPU Memory
# ----------------------------
torch.cuda.empty_cache()
gc.collect()

0

In [3]:
# ----------------------------
# Device Setup
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


In [4]:

# ----------------------------
# ResNet-18 Model Definition
# ----------------------------
class ResNet18_Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = models.resnet18(weights="IMAGENET1K_V1")
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

In [13]:
# ----------------------------
# Dataset (Memory Safe)
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),   
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset   = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [14]:
# ----------------------------
# Model Setup
# ----------------------------
num_classes = 10
model = ResNet18_Model(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
# ----------------------------
# FLOPs & Params
# ----------------------------
macs, params = get_model_complexity_info(model, (3, 64, 64),
                                         as_strings=True,
                                         print_per_layer_stat=False,
                                         verbose=False)
macs_value = macs.replace('Mac', '').strip()
if 'G' in macs_value:
    macs_num = float(macs_value.replace('G', '')) * 2
    flops_per_forward = f"{macs_num:.2f} GFLOPs"
elif 'M' in macs_value:
    macs_num = float(macs_value.replace('M', '')) / 1000 * 2
    flops_per_forward = f"{macs_num:.3f} GFLOPs"
else:
    flops_per_forward = f"{macs_value} MACs"

print(f"FLOPs per forward pass: {flops_per_forward} | Params: {params}")


FLOPs per forward pass: 0.298 GFLOPs | Params: 11.18 M


In [16]:
# ----------------------------
# Train & Validate
# ----------------------------
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)

def validate(model, loader, criterion):
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    acc = 100 * correct / total
    return val_loss / total, acc

In [17]:
# ----------------------------
# Energy & CO₂ Tracking
# ----------------------------
output_dir = "./emissions_resnet18_no_tuning"
os.makedirs(output_dir, exist_ok=True)

tracker = EmissionsTracker(
    project_name="ResNet18_CIFAR10_NoTuning",
    output_dir=output_dir,
    measure_power_secs=1
)
tracker.start()
start_time = time.time()

[codecarbon WARNING @ 00:55:44] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 00:55:44] [setup] RAM Tracking...
[codecarbon INFO @ 00:55:44] [setup] CPU Tracking...
[codecarbon WARNING @ 00:55:46] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 00:55:46] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 00:55:46] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 00:55:46] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 00:55:46] [setup] GPU Tracking...
[codecarbon INFO @ 00:55:46] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 00:55:46] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation mode

In [19]:
# ----------------------------
# Training Loop (4 Epochs)
# ----------------------------
for epoch in range(4):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/4 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")


[codecarbon INFO @ 01:17:22] Energy consumed for RAM : 0.003547 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:17:22] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:17:22] Energy consumed for All CPU : 0.015100 kWh
[codecarbon INFO @ 01:17:22] Energy consumed for all GPUs : 0.021731 kWh. Total GPU Power : 39.479425537137416 W
[codecarbon INFO @ 01:17:22] 0.040378 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:17:23] Energy consumed for RAM : 0.003549 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:17:23] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:17:23] Energy consumed for All CPU : 0.015112 kWh
[codecarbon INFO @ 01:17:23] Energy consumed for all GPUs : 0.021749 kWh. Total GPU Power : 63.55045009522513 W
[codecarbon INFO @ 01:17:23] 0.040410 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:1

Epoch 1/4 | Train Loss: 0.1182 | Val Acc: 89.82%


[codecarbon INFO @ 01:22:20] Energy consumed for RAM : 0.004363 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:22:20] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:22:20] Energy consumed for All CPU : 0.018575 kWh
[codecarbon INFO @ 01:22:20] Energy consumed for all GPUs : 0.026933 kWh. Total GPU Power : 52.083055129880414 W
[codecarbon INFO @ 01:22:20] 0.049871 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:22:21] Energy consumed for RAM : 0.004365 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:22:21] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:22:21] Energy consumed for All CPU : 0.018587 kWh
[codecarbon INFO @ 01:22:21] Energy consumed for all GPUs : 0.026951 kWh. Total GPU Power : 65.40938327961318 W
[codecarbon INFO @ 01:22:21] 0.049903 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:2

Epoch 2/4 | Train Loss: 0.1007 | Val Acc: 90.19%


[codecarbon INFO @ 01:27:18] Energy consumed for RAM : 0.005178 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:27:18] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:27:18] Energy consumed for All CPU : 0.022048 kWh
[codecarbon INFO @ 01:27:18] Energy consumed for all GPUs : 0.032152 kWh. Total GPU Power : 59.100316838826814 W
[codecarbon INFO @ 01:27:18] 0.059378 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:27:19] Energy consumed for RAM : 0.005181 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:27:19] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:27:19] Energy consumed for All CPU : 0.022059 kWh
[codecarbon INFO @ 01:27:19] Energy consumed for all GPUs : 0.032170 kWh. Total GPU Power : 65.10323434851693 W
[codecarbon INFO @ 01:27:19] 0.059411 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:2

Epoch 3/4 | Train Loss: 0.0811 | Val Acc: 91.06%


[codecarbon INFO @ 01:32:17] Energy consumed for RAM : 0.005997 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:32:17] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:32:17] Energy consumed for All CPU : 0.025533 kWh
[codecarbon INFO @ 01:32:17] Energy consumed for all GPUs : 0.037370 kWh. Total GPU Power : 57.975675075663354 W
[codecarbon INFO @ 01:32:17] 0.068900 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:32:18] Energy consumed for RAM : 0.006000 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:32:18] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:32:18] Energy consumed for All CPU : 0.025544 kWh
[codecarbon INFO @ 01:32:18] Energy consumed for all GPUs : 0.037388 kWh. Total GPU Power : 65.35341248299936 W
[codecarbon INFO @ 01:32:18] 0.068932 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:3

Epoch 4/4 | Train Loss: 0.0722 | Val Acc: 90.58%


In [20]:
# ----------------------------
# Stop Tracker & Show Results
# ----------------------------
emissions = tracker.stop()
elapsed = time.time() - start_time

csv_file = os.path.join(output_dir, "emissions.csv")
df = pd.read_csv(csv_file)
energy = df['energy_consumed'].iloc[-1]  # kWh

[codecarbon INFO @ 01:37:39] Energy consumed for RAM : 0.006880 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:37:39] Delta energy consumed for CPU with constant : 0.000004 kWh, power : 42.5 W
[codecarbon INFO @ 01:37:39] Energy consumed for All CPU : 0.029292 kWh
[codecarbon INFO @ 01:37:39] Energy consumed for all GPUs : 0.042800 kWh. Total GPU Power : 31.620543595355738 W
[codecarbon INFO @ 01:37:39] 0.078972 kWh of electricity and 0.000000 L of water were used since the beginning.


In [21]:
print("\n==============================")
print("✅ Final Results Summary (ResNet-18)")
print("==============================")
print(f"CO₂ Emissions : {emissions:.6f} kg")
print(f"Energy Used   : {energy:.6f} kWh")
print(f"Time Taken    : {elapsed/60:.2f} min")
print(f"FLOPs         : {flops_per_forward}")
print("==============================")


✅ Final Results Summary (ResNet-18)
CO₂ Emissions : 0.010957 kg
Energy Used   : 0.078972 kWh
Time Taken    : 41.89 min
FLOPs         : 0.298 GFLOPs
